<a href="https://colab.research.google.com/github/vgitclt/ECE57000/blob/main/Copy_of_react_agent_with_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/agent/react_agent_with_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReAct Agent with Query Engine (RAG) Tools

In this section, we show how to setup an agent powered by the ReAct loop for financial analysis.

The agent has access to two "tools": one to query the 2021 Lyft 10-K and the other to query the 2021 Uber 10-K.

We try two different LLMs:

- gpt-3.5-turbo
- gpt-3.5-turbo-instruct

Note that you can plug in any LLM that exposes a text completion endpoint.

## Build Query Engine Tools

In [ ]:
%pip install llama-index-llms-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [ ]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

Download Data

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2025-03-07 20:51:04--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.07s   

2025-03-07 20:51:05 (26.4 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2025-03-07 20:51:05--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connec

In [ ]:
%pip install llama-index-readers-file
%pip install llama-index-embeddings-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.1 MB/s eta 0:00:00


In [ ]:
import openai
openai.api_key = "xxxx"
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Setup ReAct Agent

Here we setup two ReAct agents: one powered by standard gpt-3.5-turbo, and the other powered by gpt-3.5-turbo-instruct.

You can **optionally** specify context which will be added to the core ReAct system prompt.

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [ ]:
# [Optional] Add Context
# context = """\
# You are a stock market sorcerer who is an expert on the companies Lyft and Uber.\
#     You will answer questions about Uber and Lyft as in the persona of a sorcerer \
#     and veteran stock market investor.
# """
llm = OpenAI(model="gpt-3.5-turbo")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

In [ ]:
response = agent.chat("What was Lyft's revenue growth in 2021?")
print(str(response))

> Running step ec591e6e-0a52-40d0-8b97-4fdb96d68915. Step input: What was Lyft's revenue growth in 2021?
Thought: The user is asking about Lyft's revenue growth in 2021. I can use the Lyft financial tool to find this information.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: Lyft's revenue growth in 2021 was 36%.
> Running step 475c35e5-6b28-4527-a977-0b0d8a7d55f0. Step input: None
Thought: I can answer the user's question without using any more tools.
Answer: Lyft's revenue growth in 2021 was 36%.
Lyft's revenue growth in 2021 was 36%.


## Run Some Example Queries

We run some example queries using the agent, showcasing some of the agent's abilities to do chain-of-thought-reasoning and tool use to synthesize the right answer.

We also show queries.

In [ ]:
response = agent.chat(
    "Compare and contrast the revenue growth of Uber and Lyft in 2021, then"
    " give an analysis"
)
print(str(response))

> Running step bd7d0c0a-8ec9-4733-a271-f4d4f69e8bc6. Step input: Compare and contrast the revenue growth of Uber and Lyft in 2021, then give an analysis
Thought: I need to use the tools to gather information on Uber's revenue growth in 2021 before I can compare and analyze the revenue growth of Uber and Lyft.
Action: uber_10k
Action Input: {'input': "Please provide information on Uber's revenue growth in 2021"}
Observation: Uber's revenue in 2021 increased by $6.3 billion, representing a growth of 57% compared to the previous year. This growth was primarily driven by a 56% increase in Gross Bookings, with Delivery Gross Bookings growing by 71% and Mobility Gross Bookings growing by 38%. The increase in revenue was also attributed to higher demand for food delivery orders and increased trip volumes as the business recovered from the impacts of COVID-19. Additionally, revenue growth was supported by an increase in Freight revenue due to the acquisition of Transplace.
> Running step 11cf6

**Async execution**: Here we try another query with async execution

In [ ]:
# Try another query with async execution

import nest_asyncio

nest_asyncio.apply()

response = await agent.achat(
    "Compare and contrast the risks of Uber and Lyft in 2021, then give an"
    " analysis"
)
print(str(response))

> Running step a9a9720e-870c-493b-9485-62c74353d737. Step input: Compare and contrast the risks of Uber and Lyft in 2021, then give an analysis
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: In 2021, both Uber and Lyft faced similar risks related to regulatory challenges, competition, and the impact of the COVID-19 pandemic on their operations. Regulatory changes in various markets could affect their ability to operate or increase operating costs. Competition from other ride-sharing and delivery services also posed a risk to both companies, impacting market share and pricing strategies. The ongoing pandemic continued to impact travel demand and consumer behavior, affecting the overall performance of both companies. However, Uber's diversification into the delivery segment provided some resilience compared to Lyft's focus primarily on ride-sharing.
In 2021, both Uber and Lyft faced similar risks related to regulatory challenges, compet

### Compare gpt-3.5-turbo vs. gpt-3.5-turbo-instruct

We compare the performance of the two agents in being able to answer some complex queries.

#### Taking a look at a turbo-instruct agent

In [ ]:
llm_instruct = OpenAI(model="gpt-3.5-turbo-instruct")
agent_instruct = ReActAgent.from_tools(
    query_engine_tools, llm=llm_instruct, verbose=True
)

In [ ]:
response = agent_instruct.chat("What was Lyft's revenue growth in 2021?")
print(str(response))

> Running step 050052d0-6962-4f90-9498-075bb40c1e6f. Step input: What was Lyft's revenue growth in 2021?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: Lyft's revenue growth in 2021 was 36%.
> Running step 5ccffa2a-87c2-4728-b7de-12bfbb14fab6. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Lyft's revenue growth in 2021 was 36%.
Lyft's revenue growth in 2021 was 36%.


#### Try more complex queries

We compare gpt-3.5-turbo with gpt-3.5-turbo-instruct agents on more complex queries.

In [ ]:
response = agent.chat(
    "Compare and contrast the revenue growth of Uber and Lyft in 2021, then"
    " give an analysis"
)
print(str(response))

> Running step 5075e56f-e8df-4718-9627-e6d006aac6b3. Step input: Compare and contrast the revenue growth of Uber and Lyft in 2021, then give an analysis
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: In 2021, Uber's revenue growth was 57%, while Lyft's revenue growth was 36%. Uber experienced a higher revenue growth rate compared to Lyft, indicating a stronger performance in terms of revenue generation. This difference could be attributed to factors such as the scale of operations, market share, and business strategies employed by each company. Additionally, Uber's growth was driven by a significant increase in Gross Bookings, particularly in the Delivery segment, showcasing the impact of diversification in revenue streams.
In 2021, Uber's revenue growth was 57%, while Lyft's revenue growth was 36%. Uber experienced a higher revenue growth rate compared to Lyft, indicating a stronger performance in terms of revenue generation. This di

In [ ]:
response = agent_instruct.chat(
    "Compare and contrast the revenue growth of Uber and Lyft in 2021, then"
    " give an analysis"
)
print(str(response))

> Running step 1c8eee9f-4753-4648-b6b7-ac23e52196f2. Step input: Compare and contrast the revenue growth of Uber and Lyft in 2021, then give an analysis
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: Lyft's revenue growth in 2021 was 36%.
> Running step 41131190-0e0d-4511-8f58-dd81a83944bc. Step input: None
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: uber_10k
Action Input: {'input': "What was Uber's revenue growth in 2021?"}
Observation: Uber's revenue grew by 57% in 2021.
> Running step 195c443b-cb4a-4f34-82d0-e68efe3e270e. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: In 2021, Lyft's revenue growth was 36%, while Uber's revenue growth was 57%. This shows that Uber had a higher revenue grow

In [ ]:
response = agent.chat(
    "Can you tell me about the risk factors of the company with the higher"
    " revenue?"
)
print(str(response))

> Running step 6c36d504-76ed-4e8e-9859-f26f618b68e4. Step input: Can you tell me about the risk factors of the company with the higher revenue?
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The company with the higher revenue growth in 2021 was Uber. Some of the risk factors that Uber faced in 2021 included regulatory challenges, competition in the ride-sharing and delivery markets, the impact of the COVID-19 pandemic on travel demand, and potential legal and operational risks associated with its global operations. Additionally, Uber's expansion into new markets and services could introduce risks related to market acceptance and regulatory compliance.
The company with the higher revenue growth in 2021 was Uber. Some of the risk factors that Uber faced in 2021 included regulatory challenges, competition in the ride-sharing and delivery markets, the impact of the COVID-19 pandemic on travel demand, and potential legal and operational r

In [ ]:
response = agent_instruct.query(
    "Can you tell me about the risk factors of the company with the higher"
    " revenue?"
)
print(str(response))

> Running step 5134273e-1446-4bf4-962d-5b39016948ca. Step input: Can you tell me about the risk factors of the company with the higher revenue?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: lyft_10k
Action Input: {'input': 'What are the risk factors of the company with the higher revenue?'}
Observation: The risk factors of the company with the higher revenue include challenges related to forecasting revenue and managing expenses, attracting and retaining qualified drivers and riders cost-effectively, complying with laws and regulations, managing capital expenditures for current and future offerings, developing and maintaining assets like the network of vehicles, responding to macroeconomic changes, enhancing reputation and brand value, managing growth effectively, expanding geographically, hiring and retaining talented personnel, developing new platform features, and optimizing real estate portfolio.
> Running step d

**Observation**: The turbo-instruct agent seems to do worse on agent reasoning compared to the regular turbo model. Of course, this is subject to further observation!